## Imports

In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from datetime import datetime

## Load `train.csv`

In [21]:
df_train: pd.DataFrame = pd.read_csv("data/train.csv")

df_test: pd.DataFrame = pd.read_csv("data/test_public.csv")

In [22]:
df_train.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


## Delete rows where `MISSING_DATA` is true and delete column `MISSING_DATA`

In [23]:
df_train.drop(df_train[df_train["MISSING_DATA"] == True].index, inplace=True)
df_train.drop(labels="MISSING_DATA", axis=1, inplace=True)

df_test.drop(df_test[df_test["MISSING_DATA"] == True].index, inplace=True)
df_test.drop(labels="MISSING_DATA", axis=1, inplace=True)

## Delete column `DAY_TYPE`

In [24]:
df_train.drop(labels="DAY_TYPE", axis=1, inplace=True)

df_test.drop(labels="DAY_TYPE", axis=1, inplace=True)

## Delete column `TRIP_ID`

In [25]:
df_train.drop(labels="TRIP_ID", axis=1, inplace=True)

df_test.drop(labels="TRIP_ID", axis=1, inplace=True)

## Set `ORIGIN_STAND` and `ORIGIN_CALL` to 0 for entries with null `ORIGIN_STAND`

In [26]:
df_train.at[pd.isna(df_train["ORIGIN_CALL"]) == True, "ORIGIN_CALL"] = 0
df_train.at[pd.isna(df_train["ORIGIN_STAND"]) == True, "ORIGIN_STAND"] = 0

df_test.at[pd.isna(df_test["ORIGIN_CALL"]) == True, "ORIGIN_CALL"] = 0
df_test.at[pd.isna(df_test["ORIGIN_STAND"]) == True, "ORIGIN_STAND"] = 0

# df_train.drop(labels="ORIGIN_CALL", axis=1, inplace=True)
# df_test.drop(labels="ORIGIN_CALL", axis=1, inplace=True)

In [27]:
# NOTE: EMBEDDING INFO STUFF
# LE_ORIGIN_CALL = LabelEncoder() DO NOT READD, TEST SET HAS NEW ORIGIN_CALL'S
# LE_ORIGIN_CALL.fit(df_train["ORIGIN_CALL"])
# df_train["ORIGIN_CALL"] = LE_ORIGIN_CALL.transform(df_train["ORIGIN_CALL"])
LE_ORIGIN_STAND = LabelEncoder()
LE_ORIGIN_STAND.fit(df_train["ORIGIN_STAND"])
df_train["ORIGIN_STAND"] = LE_ORIGIN_STAND.transform(df_train["ORIGIN_STAND"])


# df_test["ORIGIN_CALL"] = LE_ORIGIN_CALL.transform(df_test["ORIGIN_CALL"])
df_test["ORIGIN_STAND"] = LE_ORIGIN_STAND.transform(df_test["ORIGIN_STAND"])

## One-hot encoding for `CALL_TYPE`

In [28]:
# one-hot encoding
df_train = df_train.join(pd.get_dummies(df_train["CALL_TYPE"]))
df_train.drop(labels="CALL_TYPE", axis=1, inplace=True)

df_test = df_test.join(pd.get_dummies(df_test["CALL_TYPE"]))
df_test.drop(labels="CALL_TYPE", axis=1, inplace=True)

## Assign indices to TAXI_ID for embeddings

In [29]:
# TODO: set taxi id to indices
LE_TAXI_ID = LabelEncoder()
LE_TAXI_ID.fit(df_train["TAXI_ID"])
df_train["TAXI_ID"] = LE_TAXI_ID.transform(df_train["TAXI_ID"])

df_test["TAXI_ID"] = LE_TAXI_ID.transform(df_test["TAXI_ID"])

In [30]:
df_train.head()

,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,POLYLINE,A,B,C
0,0.0,0,367,1372636858,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",0,0,1
1,0.0,7,371,1372637303,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",0,1,0
2,0.0,0,204,1372636951,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",0,0,1
3,0.0,0,330,1372636854,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",0,0,1
4,0.0,0,217,1372637091,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",0,0,1


## Convert `TIMESTAMP` column to date + time + day of week

In [31]:
df_train["YEAR"] = df_train["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).year)
df_train["MONTH"] = df_train["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).month)
df_train["DAY"] = df_train["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).day)
df_train["WEEKDAY"] = df_train["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).weekday()) # WHY ARE YOU LIKE THIS??? ALL THE OTHERS ARE FIELDS
df_train["HR"] = df_train["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).hour)
df_train["MIN"] = df_train["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).minute)
df_train.drop(labels="TIMESTAMP", axis=1, inplace=True)

df_test["YEAR"] = df_test["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).year)
df_test["MONTH"] = df_test["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).month)
df_test["DAY"] = df_test["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).day)
df_test["WEEKDAY"] = df_test["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).weekday())
df_test["HR"] = df_test["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).hour)
df_test["MIN"] = df_test["TIMESTAMP"].apply(lambda x: datetime.fromtimestamp(x).minute)
df_test.drop(labels="TIMESTAMP", axis=1, inplace=True)


## Convert `POLYLINE` column to `TARGET` column.

In [32]:
df_train["TARGET"] = df_train["POLYLINE"].apply(lambda x : 15 * max(x.count("[") - 1, 0))
df_train.drop(labels="POLYLINE", axis=1, inplace=True)

# wtf am i doing
# df_test["TARGET"] = df_test["POLYLINE"].apply(lambda x : 15 * max(x.count("[") - 1, 0))
# df_test.drop(labels="POLYLINE", axis=1, inplace=True)

## Get information for embeddings later

In [33]:
print(f"CALL_TYPE: 3")
# print(f"ORIGIN_CALL: {len(df_train['ORIGIN_CALL'].unique())}")
print(f"ORIGIN_STAND: {len(df_train['ORIGIN_STAND'].unique())}")
print(f"TAXI_ID: {len(df_train['TAXI_ID'].unique())}")

CALL_TYPE: 3
ORIGIN_STAND: 64
TAXI_ID: 448


## Export

In [34]:
df_train.head()
# df_test.head()

,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,A,B,C,YEAR,MONTH,DAY,WEEKDAY,HR,MIN,TARGET
0,0.0,0,367,0,0,1,2013,6,30,6,17,0,345
1,0.0,7,371,0,1,0,2013,6,30,6,17,8,285
2,0.0,0,204,0,0,1,2013,6,30,6,17,2,975
3,0.0,0,330,0,0,1,2013,6,30,6,17,0,645
4,0.0,0,217,0,0,1,2013,6,30,6,17,4,435


In [35]:
df_test.head()

,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,A,B,C,YEAR,MONTH,DAY,WEEKDAY,HR,MIN
0,0.0,15,342,0,1,0,2014,8,14,3,10,57
1,0.0,57,78,0,1,0,2014,8,14,3,10,50
2,0.0,15,239,0,1,0,2014,8,14,3,10,49
3,0.0,53,309,0,1,0,2014,8,14,3,10,58
4,0.0,18,393,0,1,0,2014,8,14,3,10,59


### Remove outliers

In [36]:
df_train = df_train[df_train["TARGET"] > 30]
df_train = df_train[df_train["TARGET"] < df_train["TARGET"].mean() + 5 * df_train["TARGET"].std()]

### [0, 1] Normalization

In [37]:
df_train["YEAR"] -= 2013
df_test["YEAR"] -= 2013

In [39]:
df_train.head()

,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,A,B,C,YEAR,MONTH,DAY,WEEKDAY,HR,MIN,TARGET
0,0.0,58,197,0,1,0,0,9,20,4,19,23,330
1,55972.0,0,411,1,0,0,0,7,8,0,12,14,300
2,0.0,13,405,0,1,0,0,7,24,2,6,6,600
3,0.0,0,97,0,0,1,0,12,26,3,18,10,630
4,0.0,0,255,0,0,1,0,8,20,1,18,52,675


#### Split training set

In [38]:
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_valid = df_train[int(0.9 * len(df_train)):]
df_train = df_train[:int(0.9 * len(df_train))]
df_train.to_csv("data/processed_trainv2.csv", index=False)
df_valid.to_csv("data/processed_validv2.csv", index=False)
df_test.to_csv("data/processed_testv2.csv", index=False)
